# LangChain V1 - CHAIN, TOOL, and AGENT

1️⃣ CHAIN — Simple Question → Answer

CHAIN EXAMPLE (LangChain v1 - Modern LCEL)

What this does:
- Takes a question
- Sends it to the LLM
- Gets a direct answer

No tools.
No decision making.


In [18]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Load API key from .env file
load_dotenv(".env")

# Initialize LLM
# temperature=0 makes output stable and predictable
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# Create a prompt template
# {topic} will be replaced at runtime
prompt = PromptTemplate.from_template(
    "Answer in a short list (max 3 bullets): What is {topic}?"
)

# Create a CHAIN using LCEL (| operator)
# This means: prompt → LLM
chain = prompt | llm

# Run the chain
response = chain.invoke({"topic": "LangChain"})

# Print only the text answer
print(response.content)


- **Framework for LLMs**: LangChain is a framework designed to facilitate the development of applications using large language models (LLMs) by providing tools and components for integration and interaction.
- **Modular Components**: It offers modular components such as document loaders, text splitters, and chains to streamline the process of building LLM-powered applications.
- **Use Cases**: LangChain supports various use cases, including chatbots, question-answering systems, and data analysis, enabling developers to create sophisticated AI-driven solutions.


# 2️⃣ TOOL — Simple Python Function

TOOL EXAMPLE (LangChain v1 - Modern)

What this does:
- Defines a simple Python function
- Wraps it as a LangChain tool
- No LLM involved


In [6]:
from langchain_core.tools import tool

# @tool makes this function usable by agents
@tool
def add_numbers(a: int, b: int) -> int:
    """
    Add two numbers and return the result.
    """
    return a + b

# Call the tool directly
# Tools can be used without an agent
result = add_numbers.invoke({"a": 5, "b": 7})

print("Tool result:", result)

Tool result: 12


3️⃣ AGENT — LLM Decides to Use Tool

AGENT EXAMPLE (LangChain v1 - Modern)

What this does:
- LLM reads the user question
- Decides if a tool is needed
- Calls the tool automatically
- Returns the final answer

In [20]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

load_dotenv(".env")

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools when needed."),
    ("human", "{input}")
])

# Bind tools to LLM (this enables tool calling)
llm_with_tools = llm.bind_tools([add_numbers])

# Build runnable chain
chain = prompt | llm_with_tools

# Run
result = chain.invoke({
    "input": "What is 10 plus 15?"
})

# If tool was called, execute it
if result.tool_calls:
    call = result.tool_calls[0]
    output = add_numbers.invoke(call["args"])
    print(output)
else:
    print(result.content)



25


In [ ]:
LangChain v1: Multi-Tool System (QA, Summarization & Web Search) Using Runnable Pipelines

In [21]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

# 🔐 Load API keys
load_dotenv(".env")

# 🔸 Initialize LLM (modern model)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# =====================================================
# ✅ Tool 1: Simple QA Tool
# =====================================================

qa_prompt = PromptTemplate.from_template(
    "Answer clearly with short form: {question}"
)

qa_chain = qa_prompt | llm

@tool
def simple_qa(question: str) -> str:
    """Answer factual questions clearly"""
    response = qa_chain.invoke({"question": question})
    return response.content


# =====================================================
# ✅ Tool 2: Summarizer Tool
# =====================================================

summary_prompt = PromptTemplate.from_template(
    "short Summarize the following text:\n\n{text}"
)

summary_chain = summary_prompt | llm

@tool
def summarizer(text: str) -> str:
    """Summarizes input text"""
    response = summary_chain.invoke({"text": text})
    return response.content


# =====================================================
# ✅ Tool 3: Web Search Tool (Tavily)
# =====================================================

tavily_search = TavilySearchResults(max_results=3)

@tool
def web_search(query: str) -> str:
    """Search the internet for current information"""
    results = tavily_search.invoke({"query": query})
    return str(results)


# =====================================================
# 🧪 Tool usage examples
# =====================================================

qa_query = "What is LangGraph in LangChain?"

summary_text = """
LangGraph is a framework for building stateful multi-step agents using LangChain.
It uses graph-based design to model agent workflows and memory.
"""

search_query = "Latest updates on LangChain"

print("\n🧠 Simple QA Tool Output:\n", simple_qa.invoke(qa_query))
print("\n📝 Summarizer Tool Output:\n", summarizer.invoke(summary_text))
print("\n🌐 Web Search Tool Output:\n", web_search.invoke(search_query))


C:\Users\15016\AppData\Local\Temp\ipykernel_36244\1197590838.py:55: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=3)



🧠 Simple QA Tool Output:
 LangGraph in LangChain is a framework that allows users to create and manage complex workflows by connecting different language model components and data sources, enabling more sophisticated interactions and data processing.

📝 Summarizer Tool Output:
 LangGraph is a framework that enables the creation of stateful multi-step agents with LangChain, utilizing a graph-based design to represent agent workflows and memory.

🌐 Web Search Tool Output:
 [{'title': 'Release policy - Docs by LangChain', 'url': 'https://docs.langchain.com/oss/python/release-policy', 'content': 'light logo\ndark logo\n\nPython\n\n##### Reference\n\n##### Errors\n\n##### Releases\n\n##### Policies\n\n# Release policy\n\n`langchain-core`\n`langchain`\n`langchain-community`\n\n## \u200b Release cadence\n\n`1.0.x`\n`1.1.0`\n`langchain`\n`langchain-core`\n\n## \u200b API stability\n\n`langchain`\n`langchain-core`\n`2.0.0`\n`1.0.0`\n`1.1.0`\n`1.0.0`\n`1.0.1`\n\n### \u200b Stability of other pa